In [ ]:
import glob
import matplotlib.pyplot as plt
import nana
from astropy.table import Table
from astropy.io import ascii
import pandas as pd

In [ ]:
star_files = glob.glob("*.dat")
star_files.remove('kplr010536147.dat')
print(len(star_files))

In [ ]:
print(star_files)

In [ ]:
num = 0

for star in star_files:
    data = pd.read_csv(star)
    data = pd.read_csv(star, sep=r'\s+', header=None, names=['time', 'flux'])
    star = star[:13]

    #pipeline
    lc, delta_f, sampling_time = nana.star(star)
    f_max = (3 / (2*sampling_time))
    f_min = delta_f/3
    t_fit, flux_fit, weight_fit = nana.mask_vals(lc)
    freq_full, power_full = nana.pg_full(f_min, f_max, lc)
    freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
    indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
    refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
    fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
    regions, final_freqs, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit, flux_fit, weight_fit)
    a_early, a_late, b_early, b_late = nana.check_coherence(t_fit, flux_fit, weight_fit, final_freqs)
    rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit)
    delta_chi2s = nana.null_chi_squared(t_fit, flux_fit, weight_fit) - chi2s
    sharpnesses = nana.sharpness(second_derivatives, refined_power)
    

    data = Table()


    data['modes'] = final_freqs
    data['regions'] = regions
    data['corresponding_freq_in_region_A'] = refined_freq
    data['sharpness'] = sharpnesses
    data['delta_chi_squares'] = delta_chi2s
    data['rate_of_change_of_phase'] = rate_of_phase
    data['rate_of_change_of_amplitude'] = rate_of_amp
    
    ascii.write(
        data,
        star + '.csv',
        overwrite=True,
        format="csv",
        formats={
            "modes": "{:.10f}",
            "corresponding_freq_in_region_A": "{:.5f}",
            "sharpness": "{:.7e}",
            "delta_chi_squares": "{:.7e}",
            "rate_of_change_of_phase": "{:.7e}",
            "rate_of_change_of_amplitude": "{:.7e}"
        }
    )

    num = num + 1
    print(num)
    
    

In [ ]:
lc, delta_f, sampling_time = nana.star('kplr010536147')

In [ ]:
    f_max = (3 / (2*sampling_time))
    f_min = delta_f/3
    t_fit, flux_fit, weight_fit = nana.mask_vals(lc)
    freq_full, power_full = nana.pg_full(f_min, f_max, lc)
    freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
    indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
    refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)

In [ ]:
plt.scatter(freq_mini[indices], refined_power, color = 'r', marker = 'o')
plt.plot(freq_mini, power_mini, 'k.')
plt.title("star that won't work")